<a href="https://colab.research.google.com/github/Gabriela2685/Accidentes/blob/main/puntos_negros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
import folium
import gc

In [ ]:
# Cargar los datos (asegúrate de que tu base de datos tenga columnas 'latitud' y 'longitud')
data = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vQGVuLESo9dj1QzZuPoqvsQBvGT3-NIjLA34Mus7Q6yPAxvQpRbWhjsuUJoAUEThw/pub?output=csv")

In [ ]:
# Imprimir el número de filas
print(f"Número de filas: {len(data)}")


Número de filas: 321544


In [ ]:
# Eliminar filas con valores NaN en latitud y longitud
data = data.dropna(subset=['latitud', 'longitud'])

# Tomar una muestra de 10,000 filas para reducir el consumo de memoria
data_sample = data.sample(n=20000, random_state=42)

# Crear un array con las coordenadas de la muestra
coordinates = data_sample[['latitud', 'longitud']].to_numpy()

# Configurar y aplicar el modelo MiniBatchKMeans para clústeres
num_clusters = 50  # Define el número de clústeres
kmeans = MiniBatchKMeans(n_clusters=num_clusters, batch_size=10000, random_state=42)
data_sample['cluster'] = kmeans.fit_predict(coordinates)

# Liberar memoria no utilizada
gc.collect()

# Visualizar el número de clústeres encontrados
print(f"Número de clústeres especificados: {num_clusters}")

# Calcular el centro de cada clúster
from shapely.geometry import MultiPoint
centroids = []
for cluster_num in range(num_clusters):
    cluster_points = coordinates[data_sample['cluster'] == cluster_num]
    cluster_center = MultiPoint(cluster_points).centroid
    centroids.append(cluster_center)

print("Centroides de los clústeres:", centroids)

# Crear un mapa centrado en la Ciudad de México
mapa = folium.Map(location=[19.4326, -99.1332], zoom_start=12)

# Agregar los puntos de accidentes al mapa
for i, row in data_sample.iterrows():
    folium.CircleMarker(
        location=(row['latitud'], row['longitud']),
        radius=5, color='blue', fill=True, fill_color='blue'
    ).add_to(mapa)

# Agregar los centroides al mapa (puntos rojos)
for centroide in kmeans.cluster_centers_:
    latitud, longitud = centroide  # Extraer las coordenadas del centroide
    folium.CircleMarker(
        location=(latitud, longitud),  # latitud, longitud del centroide
        radius=7, color='red', fill=True, fill_color='red'
    ).add_to(mapa)

# Guardar o mostrar el mapa
mapa.save("mapa_accidentes3.html")

Número de clústeres especificados: 50
Centroides de los clústeres: [<POINT (19.323 -99.225)>, <POINT (19.388 -99.036)>, <POINT (19.465 -99.144)>, <POINT (19.254 -99.12)>, <POINT (19.398 -99.166)>, <POINT (19.189 -99.021)>, <POINT (19.325 -99.139)>, <POINT (19.421 -99.177)>, <POINT (19.454 -99.08)>, <POINT (19.355 -99.009)>, <POINT (19.361 -99.205)>, <POINT (19.325 -99.174)>, <POINT (19.409 -99.069)>, <POINT (19.373 -99.115)>, <POINT (19.309 -99.067)>, <POINT (19.511 -99.151)>, <POINT (19.45 -99.125)>, <POINT (19.353 -99.078)>, <POINT (19.443 -99.195)>, <POINT (19.364 -99.274)>, <POINT (19.238 -99.165)>, <POINT (19.415 -99.095)>, <POINT (19.292 -99.02)>, <POINT (19.489 -99.198)>, <POINT (19.434 -99.147)>, <POINT (19.478 -99.117)>, <POINT (19.374 -99.019)>, <POINT (19.345 -99.05)>, <POINT (19.407 -99.206)>, <POINT (19.293 -99.134)>, <POINT (19.283 -99.223)>, <POINT (19.242 -99.079)>, <POINT (19.36 -99.157)>, <POINT (19.381 -99.076)>, <POINT (19.356 -98.992)>, <POINT (19.41 -99.145)>, <PO

In [ ]:
# Eliminar filas con valores NaN en latitud y longitud
data = data.dropna(subset=['latitud', 'longitud'])

# Tomar una muestra de 20,000 filas para reducir el consumo de memoria
data_sample = data.sample(n=20000, random_state=42)

# Crear un array con las coordenadas de la muestra
coordinates = data_sample[['latitud', 'longitud']].to_numpy()

# Configurar y aplicar el modelo MiniBatchKMeans para clústeres
num_clusters = 50  # Define el número de clústeres
kmeans = MiniBatchKMeans(n_clusters=num_clusters, batch_size=10000, random_state=42)
data_sample['cluster'] = kmeans.fit_predict(coordinates)

# Liberar memoria no utilizada
gc.collect()

# Contar el número de puntos en cada clúster
cluster_counts = data_sample['cluster'].value_counts()

# Seleccionar los 20 clústeres con más puntos (con mayor aglomeración)
top_20_clusters = cluster_counts.head(20)

# Filtrar los puntos de los 20 clústeres con mayor aglomeración
top_20_data = data_sample[data_sample['cluster'].isin(top_20_clusters.index)]

# Calcular el centro de cada clúster
centroids = []
for cluster_num in top_20_clusters.index:
    cluster_points = coordinates[data_sample['cluster'] == cluster_num]
    cluster_center = MultiPoint(cluster_points).centroid
    centroids.append(cluster_center)

print("Centroides de los clústeres:", centroids)

# Crear un mapa centrado en la Ciudad de México
mapa = folium.Map(location=[19.4326, -99.1332], zoom_start=12)

# Agregar los puntos de accidentes de los 20 clústeres más grandes al mapa
for i, row in top_20_data.iterrows():
    folium.CircleMarker(
        location=(row['latitud'], row['longitud']),
        radius=5, color='blue', fill=True, fill_color='blue'
    ).add_to(mapa)

# Agregar los centroides de los 20 clústeres más grandes al mapa (puntos rojos)
for centroide in centroids:
    latitud, longitud = centroide.x, centroide.y  # Extraer las coordenadas del centroide
    folium.CircleMarker(
        location=(latitud, longitud),  # latitud, longitud del centroide
        radius=7, color='red', fill=True, fill_color='red'
    ).add_to(mapa)

# Guardar o mostrar el mapa
mapa.save("mapa_accidentes_top20_clusters.html")


Centroides de los clústeres: [<POINT (19.45 -99.125)>, <POINT (19.421 -99.177)>, <POINT (19.325 -99.174)>, <POINT (19.443 -99.195)>, <POINT (19.361 -99.205)>, <POINT (19.254 -99.12)>, <POINT (19.388 -99.036)>, <POINT (19.323 -99.225)>, <POINT (19.325 -99.139)>, <POINT (19.355 -99.009)>, <POINT (19.356 -98.992)>, <POINT (19.374 -99.019)>, <POINT (19.238 -99.165)>, <POINT (19.189 -99.021)>, <POINT (19.434 -99.147)>, <POINT (19.415 -99.123)>, <POINT (19.478 -99.117)>, <POINT (19.415 -99.095)>, <POINT (19.36 -99.157)>, <POINT (19.382 -99.186)>]
